# Partial Fine Tuning the Deepweeds Dataset on ResNet

In this notebook, in order to classify different types of weed images based on the **Deepweeds Dataset** which contains RGB colour images of 9 weed species in Australia.
We'll use the ResNet, since it will offer a good foundation.  

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from collections import Counter

## Importing the Dataset

We'll now import the DeepWeeds data, perform train - test split and optimize it. We'll import it using tfds to skip some code.

In [ ]:
(data_train, data_val, data_test), data_info = tfds.load("deep_weeds", split=["train[:70%]", "train[70%:85%]", "train[85%:]"], as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/deep_weeds/incomplete.PQOT9S_3.0.0/deep_weeds-train.tfrecord*...:   0%|   …

Dataset deep_weeds downloaded and prepared to /root/tensorflow_datasets/deep_weeds/3.0.0. Subsequent calls will reuse this data.


In [ ]:
print(data_info)

tfds.core.DatasetInfo(
    name='deep_weeds',
    full_name='deep_weeds/3.0.0',
    description="""
    The DeepWeeds dataset consists of 17,509 images capturing eight different weed species native to Australia in situ with neighbouring flora.The selected weed species are local to pastoral grasslands across the state of Queensland.The images were collected from weed infestations at the following sites across Queensland: "Black River", "Charters Towers",  "Cluden", "Douglas", "Hervey Range", "Kelso", "McKinlay" and "Paluma".
    """,
    homepage='https://github.com/AlexOlsen/DeepWeeds',
    data_dir='/root/tensorflow_datasets/deep_weeds/3.0.0',
    file_format=tfrecord,
    download_size=469.32 MiB,
    dataset_size=469.99 MiB,
    features=FeaturesDict({
        'image': Image(shape=(256, 256, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=9),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False

In [ ]:
NUM_CLASSES = data_info.features["label"].num_classes
IMG_SIZE = 224 # Since ResNet50 was trained on 224x224 images
BATCH_SIZE = 32

print(data_info.features["label"].names)

['Chinee apple', 'Lantana', 'Parkinsonia', 'Parthenium', 'Prickly acacia', 'Rubber vine', 'Siam weed', 'Snake weed', 'Negative']


## Preprocessing and Data Augmentation

We'll now write functions to resize the images, normalize, and do some data augmentation for better training.

In [ ]:
def preprocess(image, label):
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  image = preprocess_input(image)
  return image, label

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.3),
])

def augment(image, label):
  return data_augmentation(image, training=True), label

## Build Data Pipeline

We'll now load the data into `tf.data` pipelinfe.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = data_train.map(preprocess, num_parallel_calls=AUTOTUNE).map(augment, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_data = data_val.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_data = data_test.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

## Class Weights

Since our dataset is heavily biased towards negative samples, we should implement class weights to creectly balance the model.

In [ ]:
train_labels = []

for _, label in tfds.as_numpy(data_train):
  train_labels.append(label)

train_labels = np.array(train_labels)

label_counts = Counter(train_labels)

class_weights = {
    cls: len(train_labels) / (NUM_CLASSES * count)
    for cls, count in label_counts.items()
}


## Squeeze and Excitation

We'll use a technique known as squeeze and excitiation to basically assign different weights adaptively to different channels.

In [ ]:
def se_block(x, reduction=16):
    filters = x.shape[-1]
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(filters // reduction, activation="relu")(se)
    se = layers.Dense(filters, activation="sigmoid")(se)
    se = layers.Reshape((1, 1, filters))(se)
    return layers.Multiply()([x, se])

## Building the Model

We'll now build our model architecture to fine tune. We'll first go with transfer learning.

In [ ]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights = "imagenet",
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
)

base_model.trainable = True # Transfer learning

for layer in base_model.layers[:140]:
    layer.trainable = False

In [ ]:

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,122,761 (92.02 MB)

 Trainable params: 15,509,001 (59.16 MB)

 Non-trainable params: 8,613,760 (32.86 MB)

## Training the Model

We'll now train our model.

The reason we went with partial layers and not full model is because resnet50 is too large to train.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=15, class_weight=class_weights, callbacks=[tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)])

Epoch 1/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 201s 435ms/step - accuracy: 0.4373 - loss: 2.1619 - val_accuracy: 0.7252 - val_loss: 3.9660
Epoch 2/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 171s 418ms/step - accuracy: 0.6371 - loss: 1.1285 - val_accuracy: 0.7160 - val_loss: 0.8179
Epoch 3/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 171s 418ms/step - accuracy: 0.6976 - loss: 0.7318 - val_accuracy: 0.6346 - val_loss: 0.9893
Epoch 4/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 173s 423ms/step - accuracy: 0.7283 - loss: 0.6345 - val_accuracy: 0.7160 - val_loss: 0.8907
Epoch 5/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 172s 422ms/step - accuracy: 0.7308 - loss: 0.7380 - val_accuracy: 0.7392 - val_loss: 0.9214
Epoch 6/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 172s 422ms/step - accuracy: 0.7775 - loss: 0.4630 - val_accuracy: 0.8142 - val_loss: 0.5545
Epoch 7/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 172s 422ms/step - accuracy: 0.7933 - loss: 0.4370 - val_accuracy: 0.7792 - val_loss: 0.6803
Epoch 8/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 174s 425ms/step - accuracy: 0.7912 -

## Evalutaion Metrics

We'll evaluate the model on the test set and print classification report and confusion matrix.

In [ ]:
y_true = []
y_pred = []

for images, labels in test_data:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(tf.argmax(preds, axis=1).numpy())

In [ ]:
print(classification_report(
    y_true,
    y_pred,
    target_names=data_info.features["label"].names,
    digits=4
))

                precision    recall  f1-score   support

  Chinee apple     0.7755    0.4551    0.5736       167
       Lantana     0.6784    0.8824    0.7670       153
   Parkinsonia     0.9735    0.6832    0.8029       161
    Parthenium     0.9307    0.6438    0.7611       146
Prickly acacia     0.5537    0.9821    0.7082       168
   Rubber vine     0.7662    0.8027    0.7841       147
     Siam weed     0.7336    0.9401    0.8241       167
    Snake weed     0.5978    0.8901    0.7152       182
      Negative     0.9329    0.8232    0.8747      1335

      accuracy                         0.8058      2626
     macro avg     0.7714    0.7892    0.7568      2626
  weighted avg     0.8410    0.8058    0.8085      2626



In [ ]:
cm = confusion_matrix(y_true, y_pred)
print(cm)

## Conclusion


The results indicate a solid classification performance with a accuracy of 75.8% and weighted F1-score of 0.76 indicating good precision and recall. Several weed categories have high recall and the negative class has a good precision. Thus the model is good but can be further refined.